# Example: How to talk to SkyServer

SciServer Compute can talk to other components of SciServer through a series of <em>modules</em>, one for each component. This example notebook shows how to use the <strong><code>SciServer.SkyServer</code></strong> module to access data from the Sloan Digital Sky Survey (SDSS).

You are welcome (encouraged!) to copy these examples into another folder and modify them to meet your needs. You can use them as a starting point to create your own scripts. Please do not edit this notebook directly, because your edits may be overwritten if changes to the SciServer modules require changes to these example notebooks.

To run the example Python scripts in this notebook, click in any of the Code cells below (the ones with the gray backgrounds). Click the play button at the top of the window (just below the menubar) to run the script, or pres Shift-Enter. The output of each cell's script will appear directly below the cell.

## Import modules

Like any Python modules, the SciServer modules must be imported before being used. The next code block first imports the SciServer modules you will need for this example notebook, then imports some other required modules. Comments in the code block explain what each module does. To learn how to import other modules, see the Python 3.5 import documentation (https://docs.python.org/3.5/reference/import.html), or the documentation of the module(s) you are trying to import.

In [ ]:
import SciServer                      # Collection of all SciServer modules
from SciServer import SkyServer       # Communicate between SciServer Compute and SkyServer
from SciServer import CasJobs         # Communicate between SciServer Compute and SkyServer
print('Imported SciServer modules')

from matplotlib import pyplot as plt  # pyplot: a commonly-used plotting module (https://matplotlib.org/api/pyplot_api.html)
from pprint import pprint             # pprint: print data structures in readable format (https://docs.python.org/3.5/library/pprint.html)
print('Imported other needed modules')

## Get help

At any point after the modules are imported, you can type "help (<em>name of module</em>)" to read the documentation for that module. This is true for all SciServer modules and most other modules as well. Try it below.

In [ ]:
# Read the help document for the SkyServer module
help(SkyServer)

# Read the help document for all modules in the SciServer system (uncomment by removing the # before running)
#help(SciServer)

## Get images

SkyServer's <a href='http://skyserver.sdss.org/public/en/tools/chart/navi.aspx'>Navigate</a> and <a href='http://skyserver.sdss.org/public/en/tools/chart/chartinfo.aspx'>Finding Chart</a> tools show images of the sky as seen by the Sloan Digital Sky Survey (SDSS). Both tools work by calling the SDSS Image Cutout web service to generate an image of a region of sky. The SkyServer module includes a command to call the Image Cutout service, which means that you can generate your own SDSS images as part of your scripts.

The two examples below show how to use this command, <code>SkyServer.getJpegImgCutout</code>, to retrieve a sky image around one of the most famous objects discovered by the SDSS, <a href='https://daily.zooniverse.org/2013/09/24/hannys-voorwerp/'>Hanny's Voorwerp</a>. The first code cell shows the simplest way to get an image, by specifying only the ra/dec position of the image center. The second shows how to customize the image by specifying more options.

These examples rely on the Python <a href='https://matplotlib.org/api/pyplot_api.html'>pyplot</a> library, which you imported above. For more information on the <strong><code>SkyServer.getJpegImgCutout</code></strong> module, see <a href='http://www.sciserver.org/docs/sciscript-python/SciServer.html#sciserver-skyserver-module'>its documentation</a> at sciserver.org.

In [ ]:
# Get SDSS image - simple example for Hanny's Voorwerp with only required parameters.

ra_center = 145.2658   # RA of image center as decimal degrees
dec_center = 34.7265   # Dec of image center as decimal degrees

img = SkyServer.getJpegImgCutout(ra = ra_center, dec = dec_center)  # create image using SkyServer.getJpegImgCutout

plt.imshow(img)    # show image using pyplot.imshow

In [ ]:
# Get SDSS image - customized example for Hanny's Voorwerp specifying more parameters.

# Required parameters
ra = 145.2658   # RA of image center as decimal degrees
dec_center = 34.7265   # Dec of image center as decimal degrees

# Optional parameters (see the command below for how they are sent to SkyServer.getJpegImgCutout)
pixel_scale = 0.15     # pixel scale in arcsec/pixel (default = 0.7; native SDSS = 0.4)
imgwidth = 512  # width of image in pixels (default = 512)
imgheight = 512 # height of image in pixels (default = 512)
SkyServer_DataRelease = "DR14"  # SDSS data release to use (latest is DR14)

option_list = 'SG'  # string containing one-letter codes for options - for a list, see http://www.sciserver.org/docs/sciscript-python/SciServer.html#sciserver-skyserver-module

# Optionally, you can send an SQL query, and results will be marked on the image with purple triangles
my_query = 'SELECT p.objID, p.ra, p.dec \n'
my_query += 'FROM fGetNearbyObjEq(145.2658,34.7265,1) y \n'
my_query += 'JOIN PhotoPrimary p ON y.objID=p.objID \n'
my_query += 'WHERE p.clean=1'

# SkyServer.getJpegImgCutout generates the image (described at http://www.sciserver.org/docs/sciscript-python/SciServer.html#sciserver-skyserver-module)
img = SkyServer.getJpegImgCutout(ra = ra_center, dec = dec_center, scale = pixel_scale, 
                                 width = imgwidth, height = imgheight, 
                                 dataRelease = SkyServer_DataRelease, opt = option_list, 
                                 query = my_query)


# Generate image with pyplot.imshow at a custom size
resolution = 72 # used to convert pixels to inches as required by pyplot (pixels per inch; screen = 72, print = 300)
figwidth = imgwidth / resolution
figheight = imgheight / resolution

plt.figure(figsize=(figwidth,figheight))
plt.imshow(img)

## Get data for a known object

The SkyServer <a href='http://skyserver.sdss.org/public/en/tools/quicklook/summary.aspx'>Quick Look</a> and <a href='http://skyserver.sdss.org/public/en/tools/quicklook/summary.aspx'>Explore</a> tools provide complete SDSS data about single astronomical objects (stars, galaxies, quasars, etc.). You can retrieve any or all of this information into your scripts with the <strong><code>SkyServer.objectSearch</code></strong> command.

The <code>SkyServer.objectSearch</code> command takes as input any of these five parameters for a single astronomical object (the same parameters that appear in the <a href='http://skyserver.sdss.org/public/en/tools/quicklook/summary.aspx'>Explore</a> search box):
1. SDSS object ID (<code>objID</code>, <em>e.g. 1237662301903192106</em>)
1. SDSS spectroscopic ID (<code>specObjID</code>, <em>e.g. 1889376924388583424</em>)
1. 3-part SDSS plate/MJD/fiber identifier (<code>plate</code>, <code>mjd</code>, <code>fiberID</code>, <em>e.g. 1678, 53433, 425</em>)
1. 6-part SDSS photometric identifier (<code>run</code>, <code>rerun</code>, <code>camcol</code>, <code>field</code>, <code>obj</code>, <em>e.g. 3918, 301, 3, 213, 42</em>)
1. ra/dec celestial coordiantes, in decimal degrees (<code>ra</code>, <code>dec</code>, <em>e.g. 229.52557, 42.74585</em>)

If you specify multiple values, they are parsed in order of priority as in the list above; thus <code>obJID</code> will take first priority. At present, <strong><code>SkyServer.objectSearch</code></strong> cannot accept ra/dec input as sexagesimal values, nor object IDs for SDSS component surveys like APOGEE. To request these or other new features, email the SciServer helpdesk at <a href='mailto:sciserver-helpdesk@jhu.edu'>sciserver-helpdesk@jhu.edu</a>.

In [ ]:
# This code cell searches SDSS for data for IC 2497, companion galaxy to Hanny's Voorwerp, 
## in five different ways. Uncomment each line that starts with "ic2497" to see how each method works.

# Regardless of method, the code returns an object called "ic2497" which is a list of dictionaries.
## The "pprint" library prints this object in a human-readable format.


# You can search by SDSS objId, position, or SDSS run/camcol/field
#ic2497 = SkyServer.objectSearch(objId=1237661382772195474)
ic2497 = SkyServer.objectSearch(ra=145.2671, dec=34.7328)
#ic2497 = SkyServer.objectSearch(plate=5798, mjd=56326, fiber=532)

# For objects with spectra, you can additionally search by specObjId or plate/mjd/fiber
#ic2497 = SkyServer.objectSearch(run=3704, rerun=301, camcol=3, field=91, obj=146)
#ic2497 = SkyServer.objectSearch(specObjId=6528114001053618176)


pprint(ic2497)  # Use Python's pprint module to produce human-readable output

## Search for objects

The above code cells show you how to get images and catalog data about objects once you know which objects you want. But more often, your first task will be find objects of interest from their sky positions and possibly other criteria. The SciServer.SkyServer method includes these three search functions (listed below with required parameters):

1. <strong><code>SkyServer.radialSearch(ra, dec, radius)</code></strong>: searches around a given point in the sky for a given distance, specified as <code>ra</code>, <code>dec</code>, <code>radius</code>. The ra and dec must be in decimal degrees, and the radius is in arcminutes.

1. <strong><code>SkyServer.radialSearch(min_ra, max_ra, min_dec, max_dec)</code></strong>: searches in a rectangle of sky. Specify both bounding ra values first, then both bounding dec values. Values must be in decimal degrees.

1. <strong><code>SkyServer.sqlSearch(sql)</code></strong>: submits an SQL query to the SDSS database, and returns a table of results. To learn how to write SQL queries to the SDSS database, see SkyServer's <a href="http://skyserver.sdss.org/public/en/help/howto/search/searchhowtohome.aspx">SQL tutorial</a> and <a href='http://skyserver.sdss.org/public/en/help/docs/sql_help.aspx'>SQL in SkyServer</a>. A good way to learn SQL is to modify existing samples, like the <a href="http://skyserver.sdss.org/public/en/help/docs/realquery.aspx">Sample Queries</a> listed on the SkyServer site.

All three functions return results as a Pandas DataFrame object. To learn how to work with a DataFrame, see the <a href="https://pandas.pydata.org/pandas-docs/stable/">Pandas documentation</a>. Optional parameters are available for each SkyServer search function; see the <a href="http://www.sciserver.org/docs/sciscript-python/SciServer.html#sciserver-skyserver-module">SciServer module documentation</a> for a full list.

Each of the code cells below shows one search function. Run each code cell to see what it returns.

In [ ]:
# Do a radial search of objects: Find objects near the coordiantes of cluster Abell 2255

df = SkyServer.radialSearch(ra=258.25, dec=64.05, radius=5, dataRelease=SkyServer_DataRelease)
df

In [ ]:
# Do a radial search of objects: Find objects near the coordiantes of cluster Abell 2255

# note how you can break up a long command, as long as tabs are consistent:
df = SkyServer.rectangularSearch(
    min_ra=258.1, 
    max_ra=258.4, 
    min_dec=63.8,
    max_dec=64.2)
df

In [ ]:
# Exectute an SQL query: Find objects near the coordiantes of cluster Abell 2255...
##  but select only galaxies with clean SDSS photometry


SkyServer_TestQuery = "select top 10 specobjid, ra, dec "
SkyServer_TestQuery += "from specobj "
SkyServer_TestQuery += "where class='GALAXY' and zWarning=0"

SkyServer_DataRelease = "DR14"

print('Query to be submitted:')
print(SkyServer_TestQuery)

print('\n')
print('Results:')
df = SkyServer.sqlSearch(sql=SkyServer_TestQuery, dataRelease=SkyServer_DataRelease)

df 
# NOTE: pandas formatted output occurs by default *only if* it is the last command in a cell. 
#   Otherwise, use print(df).
#print(df)

# SQL queries in SkyServer are limited: they must run in 90 seconds or less and 
## return 500,000 rows or less.
## If you need to run longer queries and/or other result formats, see the 
###  SciServer.CasJobs module (uncomment below for docs)

#help(CasJobs)